In [1]:
using Distributions
using ForwardDiff

INFO: Recompiling stale cache file C:\Users\jcarey\.julia\lib\v0.6\ForwardDiff.ji for module ForwardDiff.


In [2]:
# generate date
n = 1000
μ_true = 0.4
σ_true = 0.2
X = rand(Normal(μ_true, σ_true), n)

prior = Normal(0, 1)

Distributions.Normal{Float64}(μ=0.0, σ=1.0)

In [ ]:
ℒ(ϕ) = ϕ
function log_joint(X, prior, L)
    ϵ = rand(Normal(), L)
    log_prior = logpdf(prior)

In [ ]:
η = 0.01
epochs = 100

for i in 1:epochs
end

In [ ]:
ϵ = rand(Normal(), L)
log_prior = logpdf(prior)